<a href="https://colab.research.google.com/github/hiara05/atividade-DS/blob/main/Gerenciador_de_Estoque.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gerenciador de Estoque

Este é um programa desenvolvido em Python para gerenciar o estoque de produtos de forma prática e eficiente. Com ele, é possível cadastrar novos produtos, acompanhar quantidades disponíveis, atualizar preços e visualizar relatórios rápidos do estoque. A ideia é facilitar o controle e a organização, tornando a gestão mais ágil e confiável.



---



## main.py

In [ ]:
import streamlit as st
from cadastro import telacadastro
from venda import telavenda

st.title("Seja bem vindo à tela inicial! selecione uma das opcoes abaixo!")

pagina = st.selectbox("Ir para:", ["Inicio", "Cadastro de Produtos", "Registro de Vendas"])

if pagina == "Cadastro de Produtos":
    telacadastro()
elif pagina == "Registro de Vendas":
    telavenda()



---



## classProduto.py


In [ ]:
class Produto:
    def __init__ (self, nome, preco, quantidade):
        self.nome = nome
        self.preco = preco
        self.quantidade = quantidade

    def registro(self):
        with open('produtos.txt', 'a') as arquivo:
            arquivo.write (f'{self.nome};{self.preco};{self.quantidade}\n')



---



## classVenda.py

In [ ]:
class Venda:
    def __init__(self, data, produto_vendido, quantidade, total):
        self.data = data
        self.produto_vendido = produto_vendido
        self.quantidade = quantidade
        self.total = total

    def registrar_venda(self):
        with open('vendas.txt', 'a') as arquivo:
            arquivo.write(f'{self.data};{self.produto_vendido};{self.quantidade};{self.total}\n')




---



## cadastro.py

In [ ]:
import streamlit as st
import os
from classProduto import Produto

def telacadastro():
    st.title("Controle de estoque: CADASTRO de Produtos")
    st.markdown("___")

    nome = st.text_input('digite o nome do produto')
    preco = st.number_input ('digite o preco do produto', min_value=0.0, format="%.2f")
    quantidade = st.number_input('digite a quantidade do produto', min_value=0)

    if st.button("Cadastrar Produto"):
        if nome.strip() == "":
            st.warning("insira o nome do produto!!!")
        else:
            produto = Produto(nome, preco, quantidade)
            produto.registro()
            st.success("produto salvo com sucesso!")

    if os.path.exists("produtos.txt"):
        with open("produtos.txt", "r") as arquivo:
            for linha in arquivo:
                linha = linha.strip()
                if linha:
                    nome_produto, preco_produto, quantidade_produto = linha.split(';')
                    st.markdown('___')
                    st.write(f'Nome: {nome_produto} | Preco: {preco_produto} | Quantidade: {quantidade_produto}')



---



## venda.py

In [ ]:
import streamlit as st
import os
import datetime
from classVenda import Venda

def telavenda():
    st.title("Controle de estoque: VENDA de Produtos")
    st.markdown("___")

    data= st.date_input ("Data da venda ", value=datetime.date.today())
    produto_vendido = st.text_input ("Nome do produto vendido")
    quantidade = st.number_input ("Quantidade do produto vendido", min_value=1, step=1)
    preco_unitario = st.number_input ("Preco unitario do produto", min_value=0.0, format="%.2f")

    if st.button("Registrar Venda"):
        if produto_vendido.strip() == "":
            st.warning("Digite o nome do produto!")
        else:
            if not os.path.exists("produtos.txt"):
                st.error("Nenhum produto cadastrado ainda!")
            else:
                produtos = []
                produto_encontrado = False
                estoque_suficiente = False

                with open("produtos.txt", "r") as arquivo:
                    for linha in arquivo:
                        nome, preco, qtd = linha.strip().split(";")
                        qtd = int(qtd)
                        if nome == produto_vendido:
                            produto_encontrado = True
                            if qtd >= quantidade:
                                qtd -= quantidade
                                estoque_suficiente = True
                            else:
                                st.error(f"Estoque insuficiente! So tem {qtd} unidades.")
                        produtos.append((nome, preco, qtd))

                if not produto_encontrado:
                    st.error("Produto nao encontrado no estoque.")
                elif estoque_suficiente:
                    with open("produtos.txt", "w") as arquivo:
                        for nome, preco, qtd in produtos:
                            arquivo.write(f"{nome};{preco};{qtd}\n")

                    total = quantidade * preco_unitario
                    venda = Venda(data, produto_vendido, quantidade, total)
                    venda.registrar_venda()
                    st.success(f"Venda registrada com sucesso! Valor total: R$ {total:.2f}")

    if os.path.exists("vendas.txt"):
        st.subheader("Historico de Vendas")
        with open("vendas.txt", "r") as arquivo:
            for linha in arquivo:
                linha = linha.strip()
                if linha:
                    data_venda, produto, qtd, total = linha.split(";")
                    st.write(f"Data: {data_venda} | Produto: {produto} | Qtd: {qtd} | Total: R$ {total}")





---


# Agora, vamos modificar em partes este código para utilizarmos uma API para cadastro das pessoas!


## classCliente.py


In [ ]:
import requests

class Cliente:
    def __init__(self, cpf, nome, nascimento, cep, endereco, telefone):
        self.cpf = cpf
        self.nome = nome
        self.nascimento = nascimento
        self.cep = cep
        self.endereco = endereco
        self.telefone = telefone

    def salvar(self):
        with open("clientes.txt", "a") as arquivo:
            arquivo.write(f"{self.cpf};{self.nome};{self.nascimento};{self.cep};{self.endereco};{self.telefone}\n")

    @staticmethod
    def buscar_endereco(cep):
        url = f"https://brasilapi.com.br/api/cep/v1/{cep}"
        resposta = requests.get(url)
        if resposta.status_code == 200:
            dados = resposta.json()
            return f"{dados['street']}, {dados['neighborhood']}, {dados['city']}-{dados['state']}"
        else:
            return "Endereco nao encontrado"


## cadastro_cliente.py

In [ ]:
import streamlit as st
from classCliente import Cliente

st.header("Cadastro de Clientes")

cpf = st.text_input("CPF")
nome = st.text_input("Nome completo")
nascimento = st.date_input("Data de nascimento")
cep = st.text_input("CEP")
telefone = st.text_input("Telefone")

endereco = ""
if cep.strip() != "":
    endereco = Cliente.buscar_endereco(cep)
    st.write("Endereco encontrado:", endereco)

if st.button("Cadastrar Cliente"):
    if cpf.strip() == "" or nome.strip() == "":
        st.warning("Preencha os campos obrigatorios")
    else:
        cliente = Cliente(cpf, nome, str(nascimento), cep, endereco, telefone)
        cliente.salvar()
        st.success("Cliente cadastrado com sucesso!")

# Exibir clientes cadastrados
st.subheader("Clientes cadastrados")
try:
    with open("clientes.txt", "r") as arquivo:
        for linha in arquivo:
            dados = linha.strip().split(";")
            if len(dados) == 6:
                st.write(f"CPF: {dados[0]} | Nome: {dados[1]} | Nascimento: {dados[2]} | Endereco: {dados[4]} | Telefone: {dados[5]}")
except FileNotFoundError:
    st.info("Nenhum cliente cadastrado ainda.")


## venda.py


In [ ]:
import streamlit as st
import datetime
from classVenda import Venda

st.header("Venda de Produtos")

data = st.date_input("Data da venda", datetime.date.today())

# Selecionar cliente
clientes = []
try:
    with open("clientes.txt", "r") as arquivo:
        for linha in arquivo:
            dados = linha.strip().split(";")
            if len(dados) == 6:
                clientes.append(f"{dados[0]} - {dados[1]}")
except FileNotFoundError:
    pass

cliente_escolhido = st.selectbox("Selecione o cliente", clientes)

# Produto e venda
produto_vendido = st.text_input("Nome do produto vendido")
quantidade = st.number_input("Quantidade do produto vendido", min_value=1)
preco_unitario = st.number_input("Preco unitario do produto", min_value=0.0, format="%.2f")
total = preco_unitario * quantidade

if st.button("Registrar Venda"):
    if produto_vendido.strip() == "" or cliente_escolhido == "":
        st.warning("Preencha todos os campos")
    else:
        venda = Venda(str(data), produto_vendido, quantidade, total, cliente_escolhido)
        venda.registrar_venda()
        st.success("Venda registrada com sucesso!")

st.write(f"Total da venda: {total:.2f}")


## classVenda.py

In [ ]:
class Venda:
    def __init__(self, data, produto_vendido, quantidade, total, cliente):
        self.data = data
        self.produto_vendido = produto_vendido
        self.quantidade = quantidade
        self.total = total
        self.cliente = cliente

    def registrar_venda(self):
        with open("vendas.txt", "a") as arquivo:
            arquivo.write(f"{self.data};{self.produto_vendido};{self.quantidade};{self.total};{self.cliente}\n")


exemplo de como ficaria um arquivo JSON
## Nova seção

In [ ]:
  "cpf": "98765432100",
        "nome": "Maria Oliveira",
        "data_nascimento": "1995-08-20",
        "endereco": {
            "cep": "30140071",
            "logradouro": "Avenida Afonso Pena",
            "bairro": "Centro",
            "cidade": "Belo Horizonte",
            "estado": "MG"
        },
        "telefone": "(31) 98888-8888"